# Imports

In [ ]:
import pandas as pd
import numpy as np
from random import shuffle

# Random dataset

In [ ]:
nodes = 5
# creating a random matrix with respective probabilities
val = np.random.choice(2, size=(nodes, nodes), p=[0.3, 0.7])
val

array([[1, 0, 1, 1, 1],
       [1, 0, 0, 1, 0],
       [1, 1, 0, 0, 1],
       [1, 1, 1, 0, 0],
       [1, 0, 1, 1, 1]])

In [ ]:
# convert the ranbom matrix to a adjacency matrix by make it symmetric to diagonal
for i, row in enumerate(val):
  val[:, i] = row
np.fill_diagonal(val, 1)
data = val

# Real Data

In [ ]:
data = pd.read_excel('rnd_200_rows.xlsx')
data = np.array(data)
data[data == -1] = 0
nodes = len(data)

# LSH

In [ ]:
total = 20
band_num = 5
row_num  = total / band_num

In [ ]:
def create_hash_func(size: int):
    # function for creating the hash vector/function
    hash_ex = list(range(size))
    shuffle(hash_ex)
    return hash_ex

def build_minhash_func(vocab_size: int, nbits: int):
    # function for building multiple minhash vectors
    hashes = []
    for _ in range(nbits):
        hashes.append(create_hash_func(vocab_size))
    return hashes


In [ ]:

# we create 20 minhash vectors
minhash_funcs = build_minhash_func(len(data), total)

In [ ]:
def create_hash(minhash_fn, vector: list):
    # use this function for creating our signatures (eg the matching)
    signature = []
    for func in minhash_fn:
        for i in range(len(vector)):
            idx = func.index(i)
            signature_val = vector[idx]
            if signature_val == 1:
                signature.append(idx)
                break
    return signature

In [ ]:
# now create signatures
sigs = []
for i in range(len(data)):
  sigs.append(create_hash(minhash_funcs, data[i]))

In [ ]:
bands_clusters = []
for i in range(band_num):
  band_cluster = dict()
  for j, sig in enumerate(sigs):
    lsh = sum(sig[i*band_num: (i+1)*band_num]) % nodes
    band_cluster[lsh] = (band_cluster.get(lsh,list()))+ [j]
  bands_clusters.append(band_cluster)

In [ ]:
def score(val, cluster):
  score = 0
  for lst in cluster.values():
    for i in range(len(lst)):
      for j in range(i, len(lst)):
        
        if val[lst[i]][lst[j]] == 0:
          score -= 1
  return score


In [ ]:
def score_agree_disagree(val, cluster):
  plus_score = 0
  score = 0
  for lst in cluster.values():
    for i in range(len(lst)):

      for j in range(len(val)):

        if j not in lst:
          if val[lst[i]][j] == 1:
            plus_score -= 1
 
        for j in range(i, len(lst)):
          if val[lst[i]][lst[j]] == 0:
            score -= 1
  return score + (plus_score/2)

In [ ]:
def score_agree_disagree_print(val, cluster):
  score = 0
  plus_score = 0
  for lst in cluster.values():
    for i in range(len(lst)):

      for j in range(len(val)):

        if j not in lst:
          if val[lst[i]][j] == 1:
            plus_score -= 1
            print('+err', lst[i], j)
        for j in range(i, len(lst)):
          if val[lst[i]][lst[j]] == 0:
            score -= 1
            print('-err', lst[i], lst[j])   
  return score + (plus_score/2)

In [ ]:
score(data, bands_clusters[4])

-13054

In [ ]:
def find_best_cluster(bands_clusters):
  max = -float('inf')
  max_cluster= None
  for cluster in bands_clusters:
    # scr = score(data, cluster)
    scr = score_agree_disagree(data, cluster)
    if scr > max:
      max = scr
      max_cluster = cluster
    if scr == 0:
      break
  return max_cluster, max

## Do it many times

In [ ]:
best_cluster = None
best_score = -float('inf')

for _ in range(50):
  # we create 20 minhash vectors
  minhash_funcs = build_minhash_func(len(data), total)

  # now create signatures
  sigs = []
  for i in range(len(data)):
    sigs.append(create_hash(minhash_funcs, data[i]))

  bands_clusters = []
  for i in range(band_num):
    band_cluster = dict()
    for j, sig in enumerate(sigs):
      lsh = sum(sig[i*band_num: (i+1)*band_num].join('')) % nodes
      # print(sig[i*band_num: (i+1)*band_num])
      # lsh = int(''.join(map(str,sig[i*band_num: (i+1)*band_num])), 2) % nodes
      band_cluster[lsh] = (band_cluster.get(lsh,list()))+ [j]
    bands_clusters.append(band_cluster)
  cluster, scr = find_best_cluster(bands_clusters)
  if scr > best_score:
    best_score = scr
    best_cluster = cluster


print(best_score)
print(best_cluster)
  
score_agree_disagree_print(data, best_cluster)

[32, 72, 19, 17, 47]


ValueError: ignored

In [ ]:
print(len(best_cluster.keys()))

In [ ]:
a= [1,2,3]

In [ ]:
a.remove(1)

In [ ]:
a

In [ ]:
int(''.join(map(str,a)))